**PyTorch**

Dans ce tutoriel, on va explorer l'implémentation et l'apprentissage d'un réseau de neurones en utlisant PyTorch afin de classifier des images.
Pour ce faire, on doit compléter les étapes suivantes:
- importation et traitement des données
- implémentation du modèle
- apprentissage du modèle 
- validation du modèle

**Modèles**

Le module ``torchvision.models`` propose plusieurs modèles préformés de l'état de l'art qui permettent d'effectuer plusieurs tâches de vision par ordinateur.

Ici, on va utiliser le modèle **ResNet18**. Il s'agit d'un réseau de neurones convolutif de 18 couches de profondeur. Les couches profondes utilisent les résidus des couches précédentes, d'où vient son nom *Residual Network*. Cette technique permet d'éviter le problème de *Vanishing Gradient* qui limite la performance des réseaux profonds.

Durant l'apprentissage, le réseau traitera l'entrée à travers toutes les couches et retournera une distribution de probabilité sur toutes les classes possibles. Ensuite, on calculera une fonction de perte qui permet d'estimer l'erreur de la prédiction du réseau. Enfin, la propagation des gradients dans le réseau permettra d'optimiser les paramètres des différenetes couches.

Pour ce faire, deux fonctions sont indispensables: **forward** et **backward**.
- La fonction backward calcule les gradients des paramètres et les fait propager dans le réseau. On définit notre réseau de neurones comme une sous-classe de nn.Module. Cela permet d'hériter la fonction backward. 
- Ainsi, il suffit de définir la fonction forward qui calcule la sortie du réseau.

In [79]:
import torch.nn as nn
from torchvision import models

class ResNet(nn.Module):

  def __init__(self, class_num=2, architecture="resnet18", pretrained=True):
    super(ResNet,self).__init__()
    self.pretrained = pretrained
    model = models.resnet18(pretrained=pretrained)
    fc_input_dim = model.fc.in_features
    #change the dimension of output
    model.fc = nn.Linear(fc_input_dim, class_num)
    self.model = model
  def forward(self, x):
    x = self.model(x)
    return x


**Données**

Ici, on va utiliser une base de données qui contient des images de plats différents. Notre objectif est de faire une classfication binaire pour vérifier si une image représente une pizza. La base de données contient 1900 images, que l'on va diviser en deux ensembles: **ensemble d'apprentissage** et **ensemble de validation**.
Les images sont de type *RGB*, de largeur 384 et d'hauteur 512. 

On commence par définir des transformations de données. Dans le déploiement, le modèle sera appliqué sur des données du monde réel qui pourraient contenir du bruit. Cela réduit la performance du réseau. Pour résoudre à cela, on utilise une méthode nommée **augmentation des données**. Elle consiste à appliquer des transformations aléatoires sur les données d'apprentissage pour que l'apprentissage soit plus général. Ici, on rogne aléatoirement les images en utilisant la méthode ``TF.crop``. 

Le modèle ResNet prend en entrée des tenseurs de réels dans l'intervalle [-1, 1]. Donc, on transforme les images à des tenseurs et on les normalisent.

PyTorch propose une classe abstraite ``torch.utils.data.Dataset`` qui permet de charger et manoeuvrer la base de données. On prend usage de cette classe et donc on doit surcharger deux méthodes: ``__get_item__`` et ``__len__``.

Pour obtenir l'accès aux données et les mettre en mémoire, on utilise la classe ``torch.utils.data.DataLoader``. DataLoader dans Pytorch encapsule un ensemble de données et donne accès aux données sous-jacentes. Ce wrapper contiendra des batchs d'images par taille de batch définie.


In [ ]:
#télécharger les données
import boto3
import os
s3 = boto3.client('s3',endpoint_url='https://minio.lab.sspcloud.fr/')
s3.download_file(Bucket="mbenxsalha", Key="tutorial/pizza_not_pizza.zip", Filename="data")
!unzip data
!rm pizza_not-pizza.zip

In [87]:
import random
import os
from PIL import Image
import torchvision.transforms.functional as TF
import torch
import torch.nn.functional as F

IMG_EXTENSIONS = [
    '.jpg', '.JPG', '.jpeg', '.JPEG',
    '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP']

def is_image_file(filename):
  return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)

def load_image(path):
  img = Image.open(path).convert("RGB")
  return img

def make_dataset(root):
  pizza_path = os.path.join(root,"pizza")
  not_pizza_path = os.path.join(root,"not_pizza")
  data = []
  for img in os.listdir(pizza_path):
    path = os.path.join(pizza_path, img)
    data.append((path, 1))

  for img in os.listdir(not_pizza_path):
    path = os.path.join(not_pizza_path, img)
    data.append((path, 0))

  return data


In [88]:
from torch.utils.data import Dataset

class MyTransformer():
  def __init__(self, crop):
    self.crop = crop

  def __call__(self, img, rot=None):
    img = TF.resize(img, (256,256))
    img = TF.crop(img, self.crop[0], self.crop[1], 224, 224)
    img = TF.to_tensor(img)
    img = TF.normalize(img, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    return img

class DatasetGenerator(Dataset):
    def __init__(self, root, transform=None):
        imgs = make_dataset(root)
        self.root = root
        self.imgs = imgs
        self.transform = transform

    def __getitem__(self, index):
        path, lab = self.imgs[index]
        img = load_image(path)


        # If a custom transform is specified apply that transform
        if self.transform is not None:
            img = self.transform(img)
        else:  # Otherwise define a random one (random cropping)
            top = random.randint(0, 256 - 224)
            left = random.randint(0, 256 - 224)
            transform = MyTransformer([top, left])
            # Apply the transformation
            img = transform(img)
        return img, lab

    def __len__(self):
        return len(self.imgs)

In [89]:
from torch.utils.data import DataLoader, random_split

batch_size = 64
data_root = "shipsnet/shipsnet"

dataset = DatasetGenerator(data_root)
train_set_length = int(0.7 * len(dataset))
test_set_length = len(dataset) - train_set_length
#split the dataset 
train_set, test_set = random_split(dataset, [train_set_length, test_set_length])
#define loaders
train_loader = DataLoader(train_set, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_set, shuffle=True, batch_size=batch_size)

**Optimisation**

``torch.optim`` propose plusieurs méthodes de descente de gradient. Ici, on utilisera **la descente de gradient stochastique (SGD)**. 
On inclut aussi la méthode de **dégradation des pondérations (weight decay)** qui est une technique de régularisation servant à limiter le surapprentissage du réseau.

le module ``torch.nn`` propose plusieurs fonctions de perte. Dans ce tutoriel, on utilise **l'entropie croisée** . La minimsation de cette fonction permettra de rapprocher la distribution de probabilité apprise par le modèle à la distribution réelle. 

On utilise le l'application **cuda** de Nvidia qui permet de paralléliser le calcul en utilisant le processeur graphique (GPU). Cela permet d'optimiser le temps d'exécution.

In [91]:
import torch.optim as optim
import torch.nn.functional as F
import torch
import torch.nn as nn

class_num = 2

ce_loss=nn.CrossEntropyLoss()

#set hyperparameters
lr = 0.001
weight_decay = 0.0005

#set device to cuda if nvidia gpu is available
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
#initialize model
model = ResNet(class_num=class_num)
model = model.to(device)

#define optimizer
opt_model = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)

**Apprentissage**

Pour former le modèle, on doit boucler sur l'itérateur de données, alimenter les entrées au réseau et optimiser les paramètres. 

``torch.utils.tensorboard`` permet d'enregistrer des logs (la perte à chaque itération par exemple)  de l'apprentissage pour les visualiser après.

``tqdm`` permet d'afficher une barre de progression

In [92]:
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

epoch = 10
model.train()
torch.set_grad_enabled(True)
writer = SummaryWriter()
for epo in range(1,epoch+1):
  correct = 0
  print("Epoch {}/{} \n".format(epo, epoch))
  with tqdm(total=len(train_loader), desc="Train") as pb:
    for batch_num, (img, img_label) in enumerate(train_loader):
      opt_model.zero_grad()
      img = img.to(device) 
      img_label = img_label.to(device)
      outputs = model(img)
      loss = ce_loss(outputs, img_label)
      loss.backward()
      opt_model.step()
      correct += (torch.argmax(outputs, dim=1)==img_label).sum().item()
      writer.add_scalar("Loss_recoginition/source", loss, epo * len(train_loader) + batch_num)
      pb.update(1)

Epoch 1/10 



Train: 100%|██████████| 44/44 [00:15<00:00,  2.91it/s]


Epoch 2/10 



Train: 100%|██████████| 44/44 [00:14<00:00,  3.01it/s]


Epoch 3/10 



...

Train: 100%|██████████| 44/44 [00:14<00:00,  2.96it/s]


Epoch 9/10 



Train: 100%|██████████| 44/44 [00:14<00:00,  2.97it/s]


Epoch 10/10 



Train: 100%|██████████| 44/44 [00:14<00:00,  3.09it/s]


**Evaluation**

La dernière étape consiste à évaluer le modèle sur l'ensemble de test.
Avant de commencer, on met le modèle sur le mode d'évaluation avec ``model.eval()`` pour geler les paramètres.

In [93]:
from tqdm import tqdm
correct, num_predictions = 0, 0
with tqdm(total=len(test_loader), desc="Test") as pb:
  model.eval()
  for batch_num, (img, img_label) in enumerate(test_loader):
    img = img.to(device)
    img_label = img_label.to(device)
    predictions = model(img)
    correct += (torch.argmax(predictions, dim=1)==img_label).sum().item()
    num_predictions += predictions.shape[0]
    pb.update(1)

accuracy = correct / num_predictions
print("\n Accuracy: {}".format(accuracy))

Test: 100%|██████████| 19/19 [00:04<00:00,  4.47it/s]


 Accuracy: 0.9875


**Affichage des logs**

Tensorboard enregistre les logs dans un dossier ``runs``.
On utilise la commande suivante pour les afficher

In [65]:
%reload_ext tensorboard
%tensorboard --logdir runs

Reusing TensorBoard on port 6006 (pid 904), started 22:10:31 ago. (Use '!kill 904' to kill it.)

**Enregistrement du modèle**

Enfin, on enregistre le modèle pour l'exploiter après

In [95]:
torch.save(model.state_dict(), "model_1.pth")